In [ ]:
import numpy  as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
house_data = pd.read_csv("./data/king_county.csv") # cargamos fichero
print(house_data.shape)
house_data.head(5).T                                 # visualizamos 5 primeras filas